# RAG 통합 최종 버전
## 다현님 + 나현님 + 사용자 작업 통합

이 노트북은 세 분의 작업을 통합한 최종 버전입니다:
- **다현님 작업**: 본문/부칙 분리, 학위/정원 문장 생성, 본문 청킹
- **나현님 작업**: UpstageDocumentParseLoader 활용 (참고)
- **사용자 작업**: 표 데이터 처리 및 벡터화

### 통합 전략
1. PDF 본문과 부칙을 분리하여 청킹
2. CSV 표 데이터를 문장으로 변환
3. 모든 문서를 FAISS 벡터 스토어에 통합
4. RAG 파이프라인 구축 및 평가


In [1]:
# 환경 설정
import os
import sys
from pathlib import Path
from typing import List, Optional

import pandas as pd
import re

# Upstage API Key 설정
os.environ.setdefault("UPSTAGE_API_KEY", "up_g01lLro3nefoqy9IZGw3NK4ExjfIQ")
UPSTAGE_API_KEY = os.environ.get("UPSTAGE_API_KEY", "")

# 작업 디렉토리 설정
CURRENT_DIR = Path(__file__).parent if "__file__" in globals() else Path.cwd()
if Path.cwd() != CURRENT_DIR:
    os.chdir(CURRENT_DIR)


print(f"✅ 작업 디렉토리: {CURRENT_DIR}")
print(f"✅ Upstage API Key: {UPSTAGE_API_KEY[:20]}...")


✅ 작업 디렉토리: c:\Users\user\-NLP-RAG-Project
✅ Upstage API Key: up_g01lLro3nefoqy9IZ...


In [2]:
# 필요한 라이브러리 설치
%pip install langchain_upstage langchain_community langchain-openai openai pdfplumber pandas faiss-cpu wikipedia-api


Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

PDF_PATH = CURRENT_DIR / "ewha.pdf"

if not PDF_PATH.exists():
    raise FileNotFoundError(f"❌ PDF 파일을 찾을 수 없습니다: {PDF_PATH}")

print("📄 PDF 파일 로딩 중...")
loader = PyPDFLoader(str(PDF_PATH))
docs = loader.load()

# 부칙 시작 전까지만 선택 (38page 기준)
docs = [d for d in docs if int(d.metadata["page"]) < 38]

print(f"✅ 본문 페이지 로드 완료: {len(docs)} 페이지")

full_text = "\n".join(d.page_content for d in docs)


# -------------------------------------------------------------
# 부칙("부칙") 등장 인덱스 찾기
# -------------------------------------------------------------
split_index = full_text.find("부칙")

if split_index == -1:
    print("⚠️ '부칙' 문자열을 찾을 수 없습니다. 전체 본문만 처리합니다.")
    main_text = full_text
    appendix_text = ""
else:
    main_text = full_text[:split_index]
    appendix_text = full_text[split_index:]

print(f"📄 본문 길이: {len(main_text)}")
print(f"📄 부칙 길이: {len(appendix_text)}")


# =============================================================
# 2. 본문 1차 구조 기반 분리
# =============================================================
def split_by_structure(text: str):
    """
    이화 학칙 문서를 조항/항목 기반으로 1차 분리
    """
    pattern = r"""
        (?=제\d+조)               # 제1조
        |(?=\n\d+\.)              # 1.
        |(?=\n\d+\))              # 1)
        |(?=\n\d+\s*-\s*\d+\s*-\s*\d+)  # 2 - 2 - 39
    """
    parts = re.split(pattern, text, flags=re.VERBOSE)
    return [p.strip() for p in parts if p.strip()]


structured_sections = split_by_structure(main_text)
print("📌 본문 1차 구조 분리 수:", len(structured_sections))


# =============================================================
# 3. 본문 RecursiveCharacterTextSplitter 청킹
# =============================================================
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200,
    separators=["별표", "\n\n제", "\n제", "\n\n", "\n", " "]  # 우선순위 높은 순
)

main_chunks = []
for sec in structured_sections:
    split_docs = text_splitter.create_documents([sec])
    main_chunks.extend(split_docs)

print("📌 최종 본문 청크 수:", len(main_chunks))


# =============================================================
# 4. 부칙 청킹
# =============================================================

# --- 4-1) 부칙을 '부칙(YYYY.MM.DD 개정)' 단위로 분리
def split_appendix(appendix: str):
    pattern = r"(?=부칙\(\d{4}\.\s*\d{1,2}\.\s*\d{1,2}\s*개정\))"
    parts = re.split(pattern, appendix)
    return [p.strip() for p in parts if p.strip()]

appendix_sections = split_appendix(appendix_text)
print("📌 1차 부칙 개정 단위 수:", len(appendix_sections))


# --- 4-2) 각 부칙 내 조항 기반 추가 분리
def split_by_articles(text: str):
    pattern = r"(?=제\d+조)|(?=\n?\d+\s*\()"
    parts = re.split(pattern, text)
    return [p.strip() for p in parts if p.strip()]

fine_grained_sections = []
for sec in appendix_sections:
    fine_grained_sections.extend(split_by_articles(sec))

print("📌 2차 부칙 조항 분리 수:", len(fine_grained_sections))


# --- 4-3) 최종 부칙 청킹
appendix_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200,
    separators=["\n\n", "\n", ". ", " "]
)

appendix_chunks = appendix_splitter.create_documents(fine_grained_sections)

print("📌 최종 부칙 청크 수:", len(appendix_chunks))


# =============================================================
# 5. 결과 요약
# =============================================================
print("\n================ RESULT ================")
print(f"본문 청크 수: {len(main_chunks)}")
print(f"부칙 청크 수: {len(appendix_chunks)}")
print(f"총 청크 수: {len(main_chunks) + len(appendix_chunks)}")
print("========================================")

📄 PDF 파일 로딩 중...
✅ 본문 페이지 로드 완료: 38 페이지
📄 본문 길이: 20919
📄 부칙 길이: 20222
📌 본문 1차 구조 분리 수: 223
📌 최종 본문 청크 수: 223
📌 1차 부칙 개정 단위 수: 80
📌 2차 부칙 조항 분리 수: 250
📌 최종 부칙 청크 수: 253

================ RESULT ================
본문 청크 수: 223
부칙 청크 수: 253
총 청크 수: 476


In [4]:
# ==================================================================
# 2. CSV 표 데이터 처리 및 문장 생성 (다현님 + 사용자 작업 통합)
# ==================================================================

def coalesce(*values: object) -> str:
    """첫 번째 유효한 값을 반환"""
    for value in values:
        if value is None:
            continue
        if isinstance(value, float) and pd.isna(value):
            continue
        text = str(value).strip()
        if text:
            return text
    return ""

# 다현님 작업: 학위 문장 생성 방식
def build_degree_sentences_from_csv(df: pd.DataFrame, year: str = "최신 개정") -> List[str]:
    """CSV에서 학위 문장 생성 (다현님 방식)"""
    sentences = []
    for idx, row in df.iterrows():
        college = coalesce(row.get("설치대학"), row.get("대학"))
        degree = coalesce(row.get("학위_종류"), row.get("학위종류"), row.get("학사"))
        major = coalesce(row.get("학과_전공"), row.get("학과 또는 전공"), row.get("전공"))
        
        if not college or not degree or not major:
            continue
            
        sentence = f"{college}의 {major} 전공은 {degree} 학위를 수여한다. ({year})"
        sentences.append(sentence)
    return sentences

# 사용자 작업: 표 데이터 처리 함수들
def build_quota_text(row: pd.Series) -> str:
    """입학정원 텍스트 생성 (사용자 방식)"""
    year = coalesce(row.get("학년도"), "2019학년도")
    college = coalesce(row.get("대학"))
    faculty = coalesce(row.get("학부"))
    major = coalesce(row.get("학과_전공"), row.get("학과 또는 전공"))
    quota = coalesce(row.get("정원"), row.get("입학정원_명")) or "미상"
    
    prefix_parts = [year]
    if college:
        prefix_parts.append(college)
    if faculty:
        prefix_parts.append(faculty)
    prefix = " ".join(prefix_parts)
    return f"{prefix} 소속 {major}의 입학 정원은 {quota}명입니다."

def build_degree_text(row: pd.Series) -> str:
    """학위 텍스트 생성 (사용자 방식)"""
    college = coalesce(row.get("설치대학"))
    major = coalesce(row.get("학과_전공"))
    degree = coalesce(row.get("학위_종류"))
    quota = coalesce(row.get("입학정원_명"))
    period = coalesce(row.get("설치_운영기간"))
    
    sentence = f"{college} 소속 {major} 전공자에게 수여하는 학위는 {degree}입니다."
    extras = []
    if quota:
        extras.append(f"입학 정원은 {quota}명")
    if period:
        extras.append(f"설치·운영 기간은 {period}")
    if extras:
        sentence += " " + ", ".join(extras) + "입니다."
    return sentence

def build_contract_text(row: pd.Series) -> str:
    """계약학과 텍스트 생성 (사용자 방식)"""
    college = coalesce(row.get("설치대학"))
    form = coalesce(row.get("설치형태"))
    major = coalesce(row.get("학과_전공"))
    degree = coalesce(row.get("학위_종류"))
    quota = coalesce(row.get("입학정원_명"))
    period = coalesce(row.get("설치_운영기간"))
    
    parts = [f"계약학과 설치·운영 정보: 설치대학={college}"]
    if form:
        parts.append(f"설치형태={form}")
    if major:
        parts.append(f"학과/전공={major}")
    if degree:
        parts.append(f"수여 학위={degree}")
    if quota:
        parts.append(f"입학 정원={quota}명")
    if period:
        parts.append(f"설치·운영 기간={period}")
    return ", ".join(parts) + "."

# CSV 파일 처리
csv_documents = []

# 1) degrees 폴더의 파일 처리 (수동 정리된 파일 사용)
degrees_dir = CURRENT_DIR / "degrees"
degree_sentences_file = degrees_dir / "degree_sentences_all.csv의 사본"
degree_latest_file = degrees_dir / "degree_latest.csv의 사본"

# 우선순위 1: 이미 문장으로 변환된 파일 사용
if degree_sentences_file.exists():
    df_sentences = pd.read_csv(degree_sentences_file, encoding='utf-8-sig')
    for _, row in df_sentences.iterrows():
        sentence = str(row.get("sentence", "")).strip()
        if sentence:
            csv_documents.append(Document(
                page_content=sentence,
                metadata={"source": "[별표 2] 학사학위의 종류", "page": 51, "type": "degree"}
            ))
    print(f"✅ degree_sentences_all.csv 처리 완료: {len(df_sentences)}개 문장")
# 우선순위 2: degree_latest.csv를 사용하여 문장 생성
elif degree_latest_file.exists():
    df_degrees = pd.read_csv(degree_latest_file, encoding='utf-8-sig')
    # 빈 행 제거
    df_degrees = df_degrees.dropna(subset=['대학', '학사', '전공'], how='all')
    # 다현님 방식: 학위 문장 생성
    degree_sentences = build_degree_sentences_from_csv(df_degrees, "최신 개정")
    for sentence in degree_sentences:
        csv_documents.append(Document(
            page_content=sentence,
            metadata={"source": "[별표 2] 학사학위의 종류", "page": 51, "type": "degree"}
        ))
    print(f"✅ degree_latest.csv 처리 완료: {len(degree_sentences)}개 문장")
# 우선순위 3: 기존 degrees.csv 사용 (하위 호환성)
elif (CURRENT_DIR / "degrees.csv").exists():
    df_degrees = pd.read_csv(CURRENT_DIR / "degrees.csv", encoding='utf-8-sig')
    degree_sentences = build_degree_sentences_from_csv(df_degrees, "최신 개정")
    for sentence in degree_sentences:
        csv_documents.append(Document(
            page_content=sentence,
            metadata={"source": "[별표 2] 학사학위의 종류", "page": 51, "type": "degree"}
        ))
    print(f"✅ degrees.csv 처리 완료: {len(degree_sentences)}개 문장")
else:
    print("⚠️ degrees 관련 파일을 찾을 수 없습니다.")

# 2) 2019_quota.csv 처리
# ==================================================================
# 하나의 CSV(연도 포함)에서 자동으로 quota 문서 생성
# ==================================================================


csv_documents = []

# CSV 파일 경로 (파일명만 맞게 수정하면 됨)
CSV_PATH = CURRENT_DIR / "capacity.csv"

# CSV 파일 체크
if not CSV_PATH.exists():
    raise FileNotFoundError(f"❌ CSV 파일을 찾을 수 없습니다: {CSV_PATH}")

# CSV 로드
df = pd.read_csv(CSV_PATH, encoding='utf-8-sig')

required_cols = {"year", "college", "department", "major", "quota"}
if not required_cols.issubset(df.columns):
    raise ValueError(f"CSV 파일 컬럼이 부족합니다. 필요한 컬럼: {required_cols}")

# 연도 목록 추출
years = sorted(df["year"].dropna().unique())
print("📌 CSV에 포함된 연도:", years)

# 텍스트를 어떻게 구성할지 함수 정의
def build_quota_text(row):
    """정원 관련 텍스트를 자연어로 생성"""
    year = int(row["year"])
    college = str(row["college"]) if pd.notna(row["college"]) else ""
    department = str(row["department"]) if pd.notna(row["department"]) else ""
    major = str(row["major"]) if pd.notna(row["major"]) else ""
    quota = str(row["quota"]) if pd.notna(row["quota"]) else ""

    text = f"""
    {year}학년도 입학정원 정보:

    - 대학: {college}
    - 학부/학과: {department}
    - 전공: {major}
    - 정원: {quota}명
    """.strip()

    return text


# 연도별로 처리
for year in years:
    df_year = df[df["year"] == year]

    count = 0
    for _, row in df_year.iterrows():
        text = build_quota_text(row)

        csv_documents.append(Document(
            page_content=text,
            metadata={
                "source": f"[별표 1] {year}학년도 입학정원",
                "year": year,
                "college": row["college"],
                "department": row["department"],
                "major": row["major"],
                "type": "quota"
            }
        ))
        count += 1

    print(f"✅ {year}년도 처리 완료: {count}개 문서 생성")

print(f"\n🎉 총 생성된 CSV 문서 수: {len(csv_documents)}개")


# 3) contract_dept.csv 처리
contract_path = CURRENT_DIR / "contract_dept.csv"
if contract_path.exists():
    df_contract = pd.read_csv(contract_path, encoding='utf-8-sig')
    for _, row in df_contract.iterrows():
        text = build_contract_text(row)
        csv_documents.append(Document(
            page_content=text,
            metadata={"source": "[별표 3] 계약학과 설치·운영", "page": 53, "type": "contract"}
        ))
    print(f"✅ contract_dept.csv 처리 완료: {len(df_contract)}개 문서")
else:
    print("⚠️ contract_dept.csv 파일이 없습니다.")

print(f"\n📦 총 CSV 문서: {len(csv_documents)}개")
# ==================================================================
# 1980년 기준 GPA 변환 테이블 CSV → Document 생성
# ==================================================================

grade_documents = []

# CSV 파일 경로
CSV_PATH = CURRENT_DIR / "grade.csv"   # → 파일 이름 맞게 수정해도 됨

if not CSV_PATH.exists():
    raise FileNotFoundError(f"❌ CSV 파일을 찾을 수 없습니다: {CSV_PATH}")

# CSV 로드
df_grade = pd.read_csv(CSV_PATH, encoding="utf-8-sig")

# 필요한 컬럼 체크
required_cols = {"year", "grade", "gpa"}
if not required_cols.issubset(df_grade.columns):
    raise ValueError(f"CSV 파일에 필요한 컬럼 부족: {required_cols}")

print(f"📄 GPA 변환 테이블 {len(df_grade)}개 항목 로드 완료")

# 텍스트 생성 함수
def build_gpa_text(row):
    year_cond = str(row["year"])  # "from 1980" 또는 "before 1980"
    grade = str(row["grade"])
    gpa = str(row["gpa"])

    text = f"""
    GPA 변환 기준:
    - 기준 연도: {year_cond}
    - 등급(Grade): {grade}
    - 환산 점수(GPA): {gpa}
    """.strip()

    return text


# Document 생성
for _, row in df_grade.iterrows():
    text = build_gpa_text(row)

    grade_documents.append(Document(
        page_content=text,
        metadata={
            "source": "GPA 변환표",
            "year_condition": row["year"],
            "grade": row["grade"],
            "gpa": row["gpa"],
            "type": "grade_table"
        }
    ))

print(f"🎉 GPA Document 생성 완료: {len(grade_documents)}개")



✅ degree_sentences_all.csv 처리 완료: 160개 문장
📌 CSV에 포함된 연도: [np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019)]
✅ 2014년도 처리 완료: 100개 문서 생성
✅ 2015년도 처리 완료: 104개 문서 생성
✅ 2016년도 처리 완료: 107개 문서 생성
✅ 2017년도 처리 완료: 105개 문서 생성
✅ 2018년도 처리 완료: 105개 문서 생성
✅ 2019년도 처리 완료: 105개 문서 생성

🎉 총 생성된 CSV 문서 수: 626개
✅ contract_dept.csv 처리 완료: 1개 문서

📦 총 CSV 문서: 627개
📄 GPA 변환 테이블 26개 항목 로드 완료
🎉 GPA Document 생성 완료: 26개


In [5]:
# ==================================================================
# 3. 모든 문서 통합 및 FAISS 벡터 스토어 생성
# ==================================================================

from langchain_upstage import UpstageEmbeddings
from langchain_community.vectorstores import FAISS

# 임베딩 모델 로드
embeddings = UpstageEmbeddings(
    api_key=UPSTAGE_API_KEY,
    model="solar-embedding-1-large"
)
print("✅ Upstage 임베딩 모델 로드 완료")

# 모든 문서 통합 (PDF 본문/부칙 + CSV 표 데이터만)
# Wikipedia 문서는 질문마다 동적으로 검색하여 추가 (벡터 스토어에는 포함하지 않음)
all_documents = main_chunks + appendix_chunks + csv_documents
print(f"📦 벡터 스토어에 포함할 문서: {len(all_documents)}개 (Wikipedia 제외)")

# 메타데이터 추가 (본문/부칙 청크)
for i, doc in enumerate(main_chunks):
    if not hasattr(doc, 'metadata') or not doc.metadata:
        doc.metadata = {}
    doc.metadata["source"] = "ewha.pdf 본문"
    doc.metadata["type"] = "main_text"

for i, doc in enumerate(appendix_chunks):
    if not hasattr(doc, 'metadata') or not doc.metadata:
        doc.metadata = {}
    doc.metadata["source"] = "ewha.pdf 부칙"
    doc.metadata["type"] = "appendix_text"

# 유효한 문서만 필터링
valid_documents = [doc for doc in all_documents if doc.page_content and doc.page_content.strip()]
print(f"🧹 필터링 후 유효 문서 수: {len(valid_documents)}")

# 문서 타입별 통계
doc_types = {}
for doc in valid_documents:
    doc_type = doc.metadata.get("type", "unknown")
    doc_types[doc_type] = doc_types.get(doc_type, 0) + 1

print("\n📊 문서 타입별 통계:")
for doc_type, count in doc_types.items():
    print(f"  - {doc_type}: {count}개")

# FAISS 벡터 스토어 생성
# Windows 한글 경로 문제 해결: 영어 경로만 있는 디렉토리 사용
import platform
import tempfile

# Windows에서 한글 경로 문제를 피하기 위해 임시 디렉토리나 영어 경로 사용
if platform.system() == 'Windows':
    # 방법 1: 환경 변수 TEMP 사용 (보통 C:\Users\USERNAME\AppData\Local\Temp)
    temp_dir = Path(tempfile.gettempdir())
    # 프로젝트별 고유한 디렉토리명 생성
    vector_db_base = temp_dir / "rag_ewha_vectorstore"
    vector_db_base.mkdir(parents=True, exist_ok=True)
    vector_db_path = str(vector_db_base.resolve())
    print(f"📂 Windows 한글 경로 문제 해결: 임시 디렉토리 사용")
    print(f"📂 벡터 DB 경로: {vector_db_path}")
else:
    # Windows가 아닌 경우 상대 경로 사용
    VECTOR_DB_DIR = CURRENT_DIR / "vectorstore"
    VECTOR_DB_DIR.mkdir(parents=True, exist_ok=True)
    vector_db_path = str(VECTOR_DB_DIR.resolve())

# 벡터 스토어 재생성 옵션 (Wikipedia 문서 제거를 위해)
FORCE_REBUILD = True  # True로 설정하면 기존 벡터 스토어를 무시하고 새로 생성

vector_store = None
vector_db_dir = Path(vector_db_path)

if not FORCE_REBUILD and vector_db_dir.exists() and (vector_db_dir / "index.faiss").exists():
    try:
        vector_store = FAISS.load_local(
            folder_path=vector_db_path,
            embeddings=embeddings,
            allow_dangerous_deserialization=True,
        )
        print(f"\n📂 기존 벡터 DB 로드 완료")
    except Exception as exc:
        print(f"⚠️ 기존 벡터 DB 로드 실패: {exc}. 새로 생성합니다.")
        vector_store = None
else:
    if FORCE_REBUILD:
        print(f"\n🔄 벡터 스토어 재생성 모드 (Wikipedia 문서 제외)")

if vector_store is None:
    print(f"\n📦 총 {len(valid_documents)}개의 문서를 벡터화 중입니다...")
    vector_store = FAISS.from_documents(valid_documents, embeddings)
    # 영어 경로 사용 (한글 경로 문제 해결)
    vector_store.save_local(vector_db_path)
    print(f"✅ 벡터 DB 생성 완료")

print(f"\n🎯 총 KB 문서 수: {len(valid_documents)}개")


✅ Upstage 임베딩 모델 로드 완료
📦 벡터 스토어에 포함할 문서: 1103개 (Wikipedia 제외)
🧹 필터링 후 유효 문서 수: 1103

📊 문서 타입별 통계:
  - main_text: 223개
  - appendix_text: 253개
  - quota: 626개
  - contract: 1개
📂 Windows 한글 경로 문제 해결: 임시 디렉토리 사용
📂 벡터 DB 경로: C:\Users\user\AppData\Local\Temp\rag_ewha_vectorstore

🔄 벡터 스토어 재생성 모드 (Wikipedia 문서 제외)

📦 총 1103개의 문서를 벡터화 중입니다...
✅ 벡터 DB 생성 완료

🎯 총 KB 문서 수: 1103개


In [6]:
import wikipediaapi
from langchain_text_splitters import RecursiveCharacterTextSplitter

def retrieve_wikipedia_docs(query: str, lang: str = 'en', chunk_size: int = 1000, chunk_overlap: int = 100):
    docs = []
    try:
        wiki_wiki = wikipediaapi.Wikipedia(
            language=lang,
            extract_format=wikipediaapi.ExtractFormat.WIKI,
            user_agent="NLP_Project"
        )
        
        page_py = wiki_wiki.page(query)
        
        if page_py.exists() and page_py.text:
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size,
                chunk_overlap=chunk_overlap
            )
            chunks = text_splitter.split_text(page_py.text)
            
            for i, chunk in enumerate(chunks):
                if chunk and chunk.strip():
                    docs.append(Document(
                        page_content=chunk,
                        metadata={
                            "source": "wikipedia",
                            "title": page_py.title,
                            "chunk_index": i,
                            "type": "wikipedia"
                        }
                    ))
    except:
        pass
    
    return docs
def extract_wikipedia_keywords(question: str) -> List[str]:
    """
    통합 업그레이드 버전:
    - 고유명사 추출
    - 영어 기반 MMLU 지식 키워드 추출
    - 한국어 분야 매핑
    - Law / Psych / Philosophy / History 분야 자동 감지
    - 대표 Wikipedia 문서 자동 매핑
    """

    keywords = []
    q = question.lower()

    # ----------------------------------------------------
    # 1) 고유명사(인물/사건/개념) 추출 (가장 우선순위 높음)
    # ----------------------------------------------------
    proper_nouns = re.findall(r"\b[A-Z][a-zA-Z]+\b", question)
    keywords.extend(proper_nouns)

    # ----------------------------------------------------
    # 2) 기존 academic_keywords (영어 기반 매핑)
    # ----------------------------------------------------
    academic_keywords = {
        'homo': 'Human evolution',
        'sapiens': 'Human evolution',
        'australopithecus': 'Human evolution',
        'erectus': 'Human evolution',
        'habilis': 'Human evolution',
        'bipolar': 'Psychology',
        'disorder': 'Psychology',
        'manic': 'Psychology',
        'kohlberg': 'Moral development',
        'kant': 'Philosophy',
        'crime': 'Criminology',
        'aristotle': 'Philosophy',
        'jurisprudence': 'Law',
        'ethics': 'Ethics',
        'larceny': 'Law',
        'criminal': 'Law',
        'clemenceau': 'History',
        'world war': 'History',
        'du fu': 'Literature',
        'poem': 'Literature',
    }

    for key, value in academic_keywords.items():
        if key in q:
            keywords.append(value)

    # ----------------------------------------------------
    # 3) 한국어 키워드 → 분야 매핑
    # ----------------------------------------------------
    keyword_mapping_kr = {
        '심리': 'Psychology',
        '심리학': 'Psychology',
        '사회': 'Sociology',
        '사회학': 'Sociology',
        '통계': 'Statistics',
        '철학': 'Philosophy',
        '역사': 'History',
        '경제': 'Economics',
        '정치': 'Political Science',
        '법': 'Law',
        '의학': 'Medicine',
        '간호': 'Nursing',
        '문학': 'Literature',
    }

    for k, v in keyword_mapping_kr.items():
        if k in question:
            keywords.append(v)

    # ----------------------------------------------------
    # 4) 분야 감지용 트리거 단어 (Law/Psych/Philosophy/History)
    # ----------------------------------------------------
    field_triggers = {
        "law": ["law", "legal", "crime", "contract", "tort", "jurisprudence", "homicide", "larceny"],
        "psych": ["psychology", "behavior", "cognitive", "mental", "disorder", "personality"],
        "phil": ["philosophy", "ethics", "moral", "kant", "aristotle", "epistemology", "metaphysics", "logic"],
        "hist": ["history", "war", "revolution", "ancient", "empire", "dynasty", "ww1", "ww2", "cold war"],
    }

    detected_fields = set()
    for field, triggers in field_triggers.items():
        for t in triggers:
            if t in q:
                detected_fields.add(field)
                break

    # ----------------------------------------------------
    # 5) 대표 Wikipedia 문서 리스트 매핑
    # ----------------------------------------------------
    representative_docs = {
        "law": [
            "Law", "Jurisprudence", "Criminal law", "Contract", "Constitutional law",
            "International law", "Law of war", "Mens rea", "Larceny"
        ],
        "psych": [
            "Psychology", "Cognitive psychology", "Behaviorism",
            "Abnormal psychology", "Personality psychology",
            "Bipolar disorder", "Anxiety disorder", "Classical conditioning"
        ],
        "phil": [
            "Philosophy", "Ethics", "Deontological ethics", "Utilitarianism",
            "Virtue ethics", "Epistemology", "Metaphysics",
            "Immanuel Kant", "Aristotle", "Categorical imperative"
        ],
        "hist": [
            "History", "World history", "French Revolution", "American Revolution",
            "World War I", "World War II", "Cold War", "Industrial Revolution",
            "History of Europe", "History of Asia"
        ],
    }

    for field in detected_fields:
        keywords.extend(representative_docs[field])

    # ----------------------------------------------------
    # 6) 영어 기반 분야 매핑 (추가 감지)
    # ----------------------------------------------------
    english_keywords = [
        'Psychology', 'Education', 'Sociology', 'Statistics', 'Mathematics',
        'Computer Science', 'Biology', 'Chemistry', 'Physics', 'Philosophy',
        'History', 'Literature', 'Economics', 'Political Science', 'Law',
        'Medicine', 'Nursing', 'Pharmacy', 'Art', 'Music'
    ]

    for keyword in english_keywords:
        if keyword.lower() in q:
            keywords.append(keyword)

    # ----------------------------------------------------
    # 7) 중복 제거 & 고유명사 우선순위 정렬
    # ----------------------------------------------------
    keywords = list(set(keywords))

    def priority(k):
        return 0 if k in proper_nouns else 1

    keywords.sort(key=priority)

    return keywords

In [7]:
# ==================================================================
# 4. RAG 파이프라인 구성 및 실행
# ==================================================================

from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage

# 테스트셋 경로
TESTSET_PATH = CURRENT_DIR / "testset.csv"
TOP_K = 5  # 검색 문서 수 증가


# 리트리버 설정
retriever = vector_store.as_retriever(search_kwargs={"k": TOP_K})

# LLM 및 프롬프트 템플릿 설정
llm = ChatUpstage(api_key=UPSTAGE_API_KEY, model="solar-pro2")

prompt_template = PromptTemplate.from_template(
    """
    다음 문맥을 바탕으로 질문에 가장 정확하게 답변하세요.
    
    **중요 지침:**
    1. 이화여대 학칙 문서를 우선 참고하세요 (구체적인 규정)
    2. Wikipedia 문서는 일반적인 학문 지식 참고용입니다
    3. 정확히 하나의 선택지만 답변하세요
    4. 답변 형식: "정답: (X)" 또는 "(X)" 형식으로 답변
    
    
    
    질문이 선택지 형식인 경우, 정답 선택지의 문자(예: (A), (B), (C), (D), (E), (I), (J), (G), (H) 등)를 포함하여 답변하세요.
    최종 답변만 제공하세요.
    
    ---
    Question: {question}
    ---
    Context:
    {context}
    """)
chain = prompt_template | llm

print(f"✅ RAG 체인 구성 완료 (Model: solar-pro2, Top-k: {TOP_K})")

# 테스트셋 로드
def read_data(file_path: Path):
    data = pd.read_csv(file_path)
    return data["prompts"], data.get("answers")

try:
    prompts, answers = read_data(TESTSET_PATH)
    print(f"✅ 테스트 프롬프트 {len(prompts)}개 로드 완료")
except FileNotFoundError:
    print(f"❌ {TESTSET_PATH} 파일을 찾을 수 없습니다.")
    prompts, answers = [], None

# 테스트 실행 (처음 5개만)
if prompts is not None and len(prompts) > 0:
    print("\n▶ 샘플 테스트 시작...\n")
    
    for i, prompt in enumerate(prompts[:5], 1):
        # 1. 기존 벡터 스토어에서 검색
        source_docs = retriever.invoke(prompt)
       

        # 2. 질문에서 Wikipedia 키워드 추출 및 검색 (나현님 방식)
        wiki_keywords = extract_wikipedia_keywords(prompt)
        wiki_docs = []
        if wiki_keywords:
            for keyword in wiki_keywords:
                try:
                    keyword_docs = retrieve_wikipedia_docs(keyword, lang='en')
                    wiki_docs.extend(keyword_docs)
                except Exception as e:
                    # 개별 키워드 검색 실패 시 무시하고 계속 진행
                    pass
        
        # 3. 컨텍스트 생성
        context_parts = []
        if source_docs:
            context_parts.append("=== 이화여대 학칙 문서 ===\n" + "\n\n".join(doc.page_content for doc in source_docs))
        if wiki_docs:
            context_parts.append("=== Wikipedia 문서 ===\n" + "\n\n".join(doc.page_content for doc in wiki_docs))
        
        context_text = "\n\n".join(context_parts) or ""
        
        # 4. 응답 생성
        response = chain.invoke({"question": prompt, "context": context_text})
        
        print(f"Q{i}: {prompt[:60]}...")
        print(f"응답: {response.content[:150]}...")
        print(f"검색 문서: 이화여대 {len(source_docs)}개, Wikipedia {len(wiki_docs)}개 (키워드: {', '.join(wiki_keywords) if wiki_keywords else '없음'})\n")
    
    print("✅ 샘플 테스트 완료")
else:
    print("⚠️ 실행할 프롬프트가 없습니다.")


✅ RAG 체인 구성 완료 (Model: solar-pro2, Top-k: 5)
✅ 테스트 프롬프트 50개 로드 완료

▶ 샘플 테스트 시작...

Q1: QUESTION1) 재학 중인 학생이 휴학을 하려면 학기 개시일로부터 며칠 이내에 휴학을 신청하야하나요?
(...
응답: 정답: (D) 90일  

이화여대 학칙 문서 제26조 ⑥항에 명시된 내용에 따르면, 재학 중인 학생이 휴학을 신청할 경우 "학기개시일로부터 90일 이내"에 신청해야 합니다. 따라서 정답은 (D)입니다.  

(D)...
검색 문서: 이화여대 5개, Wikipedia 0개 (키워드: 없음)

Q2: QUESTION2) '재입학은 a회에 한하여 할 수 있다. 다만 제 28조제4호에 의하여 제적된 자는 제적된...
응답: 정답: (A)  
재입학 횟수(a)는 "1회"로 제30조제2항에 명시되어 있고, 제적 후 재입학 가능 기간(b)은 제28조제4호 관련 규정에서 "1년"으로 제시됩니다. 따라서 a+b = 1+1 = 2로 (A)가 정답입니다.

(A)...
검색 문서: 이화여대 5개, Wikipedia 0개 (키워드: 없음)

Q3: QUESTION3) 학생이 소속 학과 또는 전공 이외의 전공 교과목을 총장이 정하는 바에 따라 몇학점 이상 ...
응답: 정답: (C) 21학점

Context의 "6.23) ③ 학생이 소속 학과 또는 전공 이외의 전공 교과목을 총장이 정하는 바에 따라 21학점 이상 취득한 때에는 부전공을 이수한 것으로 인정한다."에 따라 정답은 (C)입니다....
검색 문서: 이화여대 5개, Wikipedia 0개 (키워드: 없음)

Q4: QUESTION4) 다음 보기의 학생들 중 제적을 당하지 않는 사람을 고르면?
(A) 팜 : 징계에 의해 퇴...
응답: 정답: (D)  
제적 사유 관련 규정에 따라:  
- (A) 퇴학처분은 제적에 해당  
- (B) 평점평균 1.2로 연속 3회 학사경고 시 제적 (학칙 제4조 ④항)  
- (C) 수업료 미납은 제적 사유 (학칙 제5조

In [8]:
# ==================================================================
# 5. 전체 테스트셋 평가 (다현님 + 나현님 방식 결합)
# ==================================================================

def extract_answer(response: str) -> Optional[str]:
    """
    응답에서 답변 추출 개선 버전 (모든 알파벳 선택지 지원: A-Z)
    - 여러 선택지가 있을 경우 가장 확실한 것 선택
    - "정답:", "Answer:", "답:" 등의 키워드 우선
    """
    # 우선순위 1: 명시적인 정답 표시 (한국어/영어)
    explicit_patterns = [
        r"정답[:\s]*\(([A-Z])\)",
        r"Answer[:\s]*\(([A-Z])\)",
        r"답[:\s]*\(([A-Z])\)",
        r"\[ANSWER\]:\s*\(([A-Z])\)",
        r"최종\s*답[:\s]*\(([A-Z])\)",
        r"Final\s*Answer[:\s]*\(([A-Z])\)",
    ]
    for pattern in explicit_patterns:
        match = re.search(pattern, response, re.IGNORECASE)
        if match:
            return match.group(1).upper()
    
    # 우선순위 2: (X) 형식 (마지막 발생) - 모든 알파벳 지원
    # 여러 개가 있으면 마지막 것 (보통 최종 답변)
    pattern2 = r"\(([A-Z])\)"
    matches = list(re.finditer(pattern2, response))
    if matches:
        return matches[-1].group(1)
    
    # 우선순위 3: 단독 문자 (마지막 발생) - 모든 알파벳 지원
    # 단어 경계에서 알파벳만 매칭 (대소문자 구분 없음)
    pattern3 = r"\b([A-Z])\b"
    matches = list(re.finditer(pattern3, response, re.IGNORECASE))
    if matches:
        return matches[-1].group(1).upper()
    
    return None

# 전체 평가 실행
if prompts is not None and len(prompts) > 0:
    print("====== RAG Evaluation (전체 테스트셋) ======\n")
    print(f"📋 총 {len(prompts)}개 문제 평가 시작...\n")
    
    all_responses = []
    predictions = []
    
    for i, prompt in enumerate(prompts, 1):
        try:
            print(f"🔄 Q{i}/{len(prompts)} 처리 중...", end=" ")
            
            # 1. 기존 벡터 스토어에서 검색
            source_docs = retriever.invoke(prompt)
            
            # 2. 질문에서 Wikipedia 키워드 추출 및 검색 (나현님 방식)
            wiki_keywords = extract_wikipedia_keywords(prompt)
            wiki_docs = []
            if wiki_keywords:
                for keyword in wiki_keywords:
                    try:
                        keyword_docs = retrieve_wikipedia_docs(keyword, lang='en')
                        wiki_docs.extend(keyword_docs)
                    except Exception as e:
                        # 개별 키워드 검색 실패 시 무시하고 계속 진행
                        pass
                
                # Wikipedia 검색 결과 수 제한 (너무 많으면 혼란)
                if len(wiki_docs) > 10:
                    wiki_docs = wiki_docs[:10]  # 최대 10개만 사용
            
            # 3. 기존 문서와 Wikipedia 문서 통합
            all_source_docs = source_docs + wiki_docs
            
            # 4. 컨텍스트 생성
            context_parts = []
            if source_docs:
                context_parts.append("=== 이화여대 학칙 문서 ===\n" + "\n\n".join(doc.page_content for doc in source_docs))
            if wiki_docs:
                context_parts.append("=== Wikipedia 문서 ===\n" + "\n\n".join(doc.page_content for doc in wiki_docs))
            
            context_text = "\n\n".join(context_parts) or ""
            
            # 5. 응답 생성
            response = chain.invoke({"question": prompt, "context": context_text})
            
            pred_raw = response.content.strip()
            pred = extract_answer(pred_raw)
            
            predictions.append(pred)
            all_responses.append({
                "question_num": i,
                "question": prompt[:100],
                "prediction": pred,
                "response": pred_raw[:200],
                "context_count": len(source_docs),
                "wikipedia_count": len(wiki_docs),
                "wikipedia_keywords": ", ".join(wiki_keywords) if wiki_keywords else ""
            })
            
            # 진행 상황 출력 (모든 문제에 대해)
            gold = answers.iloc[i-1] if answers is not None else "N/A"
            # Gold 답변에서 선택지 문자 추출 (예: (D) -> D)
            gold_match = re.search(r"\(([A-Z])\)", str(gold))
            gold_letter = gold_match.group(1) if gold_match else str(gold).strip()
            
            # 정답 여부 확인 (대소문자 구분 없이)
            is_correct = pred and str(pred).upper() == str(gold_letter).upper()
            status = "✅" if is_correct else "❌" if pred else "⚠️"
            print(f"{status} Pred: {pred}, Gold: {gold} ({gold_letter})")
            
            # 10개마다 상세 정보 출력
            if i % 10 == 0 or i == len(prompts):
                print(f"  └─ 진행률: {i}/{len(prompts)} ({i/len(prompts)*100:.1f}%)")
                print("-" * 40)
        
        except Exception as e:
            import traceback
            error_msg = str(e)
            error_trace = traceback.format_exc()
            print(f"❌ 에러 발생: {error_msg}")
            print(f"  └─ Q{i} 처리 실패, None으로 저장")
            if i == 43:  # 43번 문제 디버깅
                print(f"  └─ 상세 에러:\n{error_trace}")
            predictions.append(None)
            all_responses.append({
                "question_num": i,
                "question": prompt[:100] if prompt else "",
                "prediction": None,
                "response": f"Error: {error_msg}",
                "context_count": 0,
                "wikipedia_count": 0,
                "wikipedia_keywords": ""
            })
            # 에러가 발생해도 계속 진행
            continue
    
    # 결과 저장
    results_df = pd.DataFrame(all_responses)
    results_path = CURRENT_DIR / "rag_results_combined.csv"
    results_df.to_csv(results_path, index=False, encoding='utf-8-sig')
    print(f"\n✅ 결과 저장 완료: {results_path}")
    
    # 정확도 계산
    if answers is not None:
        correct = 0
        total = 0
        for i, (pred, gold) in enumerate(zip(predictions, answers)):
            if pred is None:
                continue
            gold_match = re.search(r"\(([A-Z])\)", str(gold))
            gold_letter = gold_match.group(1) if gold_match else str(gold).strip()
            if str(pred).upper() == str(gold_letter).upper():
                correct += 1
            total += 1
        
        if total > 0:
            accuracy = correct / total * 100
            print(f"\n📊 정확도: {correct}/{total} ({accuracy:.1f}%)")
            print(f"   - 정답: {correct}개")
            print(f"   - 오답: {total - correct}개")
            print(f"   - 미응답: {len(predictions) - total}개")
else:
    print("⚠️ 실행할 프롬프트가 없습니다.")


====== RAG Evaluation (전체 테스트셋) ======

📋 총 50개 문제 평가 시작...

🔄 Q1/50 처리 중... ✅ Pred: D, Gold: (D) (D)
🔄 Q2/50 처리 중... ✅ Pred: A, Gold: (A) (A)
🔄 Q3/50 처리 중... ✅ Pred: C, Gold: (C) (C)
🔄 Q4/50 처리 중... ✅ Pred: D, Gold: (D) (D)
🔄 Q5/50 처리 중... ✅ Pred: C, Gold: (C) (C)
🔄 Q6/50 처리 중... ✅ Pred: B, Gold: (B) (B)
🔄 Q7/50 처리 중... ✅ Pred: D, Gold: (D) (D)
🔄 Q8/50 처리 중... ✅ Pred: C, Gold: (C) (C)
🔄 Q9/50 처리 중... ✅ Pred: C, Gold: (C) (C)
🔄 Q10/50 처리 중... ✅ Pred: B, Gold: (B) (B)
  └─ 진행률: 10/50 (20.0%)
----------------------------------------
🔄 Q11/50 처리 중... ✅ Pred: B, Gold: (B) (B)
🔄 Q12/50 처리 중... ✅ Pred: B, Gold: (B) (B)
🔄 Q13/50 처리 중... ✅ Pred: C, Gold: (C) (C)
🔄 Q14/50 처리 중... ❌ Pred: A, Gold: (C) (C)
🔄 Q15/50 처리 중... ✅ Pred: A, Gold: (A) (A)
🔄 Q16/50 처리 중... ✅ Pred: B, Gold: (B) (B)
🔄 Q17/50 처리 중... ✅ Pred: B, Gold: (B) (B)
🔄 Q18/50 처리 중... ✅ Pred: D, Gold: (D) (D)
🔄 Q19/50 처리 중... ✅ Pred: D, Gold: (D) (D)
🔄 Q20/50 처리 중... ✅ Pred: D, Gold: (D) (D)
  └─ 진행률: 20/50 (40.0%)
-------------------